# Импорт библиотек

In [ ]:
!pip install -q transformers

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import numpy as np
import pandas as pd
import re
import random
import textwrap
import torch
from tqdm.notebook import tqdm
import transformers
from torch.optim import AdamW

from transformers import GPT2LMHeadModel

device = 'cuda' if torch.cuda.is_available() else 'cpu'


# Загружаем токенайзер модели
from transformers import GPT2Tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('sberbank-ai/rugpt3small_based_on_gpt2')

Downloading:   0%|          | 0.00/1.63M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.21M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/608 [00:00<?, ?B/s]

# Собираем датасеты и обрабатываем для модели

In [4]:
# Собираем датасеты в лист

path_list = []

def get_path(path=str):
  path_list.append(path)
  return path_list


# каждый новый датасет аппендим в лист, тут добавляем пути вручную
path_list = get_path('/content/drive/MyDrive/Good.txt')
path_list = get_path('/content/drive/MyDrive/Bad.txt')

In [ ]:
# Задаём Параметры для обработки и настройки обучения

batch_size = 8
max_len = 64
epochs = 10

In [5]:
# Загружаем предобученную модель (в нашем случае - сбер)

model = GPT2LMHeadModel.from_pretrained(
    'sberbank-ai/rugpt3small_based_on_gpt2',
    output_attentions = False,
    output_hidden_states = False,
)

optimizer = AdamW(model.parameters(), lr = 1e-5, eps = 1e-8)

model.to(device);

Downloading:   0%|          | 0.00/526M [00:00<?, ?B/s]

In [6]:
# Предобработка текстов для модели

def get_params(path_list):
  text_list = []
  tokens_list = []
  n_text_list = []
  sheluder_list = []

# создаём массив с текстами
  for i in path_list:
    with open(i, encoding='utf8') as f:
      text = f.read()
      text_list.append(text)
    
# создаём массив с токенизированными текстами в np.array
  for i in text_list:
    tokens = tokenizer.encode(i, add_special_tokens=True)
    tokens_list.append(np.array(tokens))

# Задаём параметры модели
  for i in tokens_list:
    n_text = len(i)//(batch_size*max_len)
    n_text_list.append(n_text)

    total_steps = n_text * epochs
    
    sheluder = transformers.get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)
    sheluder_list.append(sheluder)


  return tokens_list, n_text_list, sheluder_list

# тут батчи в тензоры превращаются
def prep_tensors(x, i, batch_size=batch_size, max_len=max_len):
    batch_ids = x[i*batch_size*max_len: (i+1)*batch_size*max_len]
    batch_ids = batch_ids.reshape(batch_size, max_len)
    batch_ids = torch.tensor(batch_ids).to(device)
    return batch_ids

# забираем предобработанный текст для модели
tokens_list, n_text_list, sheluder_list = get_params(path_list)

# Обучаем модели и сохраняем

In [27]:
# Функция, которая делает много моделей и сразу сохраняет в указанную директорию

# путь, для сохранения моделей
path_models = '/content/drive/MyDrive/Textach/model_'

def get_train_models(tokens_list, n_text_list, sheluder_list):

  for i in range(len(tokens_list)):
    for epoch in range(1, epochs+1):
      print(f'epoch {epoch}/{epochs} : training')

      model.train()

      pbar = tqdm(range(n_text_list[i]))
      
      for k in pbar:
        batch_ids = prep_tensors(tokens_list[i], k)

        model.zero_grad()
        loss, logits, _ = model(batch_ids,
                              token_type_ids=None, 
                              labels=batch_ids
                             ).values()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        sheluder_list[i].step()
      torch.save(model, path_models + str(i+1) + '.pt')

  Pass

In [28]:
# Дообучение моделей

model_list = get_train_models(tokens_list, n_text_list, sheluder_list)

epoch 1/1 : training


  0%|          | 0/10 [00:00<?, ?it/s]

epoch 1/1 : training


  0%|          | 0/50 [00:00<?, ?it/s]

# Создаём функции для генерации текста

In [41]:
# Загружаем модели

model_bad = torch.load(path_models + '1' + '.pt')
model_good = torch.load(path_models + '2' + '.pt')

In [42]:
# Функции для генерации текста

def get_good(prompt):
    prompt = tokenizer.encode(prompt, return_tensors='pt').to(device)
    out = model_congr.generate(
                         input_ids=prompt,
                         max_length=20,
                         num_beams=15,
                         do_sample=True,
                         temperature=3.,
                         top_k=50,
                         top_p=0.7,
                         no_repeat_ngram_size=2,
                         num_return_sequences=7,
                        ).cpu().numpy()
    return textwrap.fill(tokenizer.decode(out[0])+'!', 120)

def get_bad(prompt):
    prompt = tokenizer.encode(prompt, return_tensors='pt').to(device)
    out = model_news.generate(Ftext
                         input_ids=prompt,
                         max_length=20,
                         num_beams=10,
                         do_sample=True,
                         temperature=7.,
                         top_k=50,
                         top_p=0.6,
                         no_repeat_ngram_size=2,
                         num_return_sequences=7,
                        ).cpu().numpy()  
    return textwrap.fill(tokenizer.decode(out[0])+'...', 120)

print (f'Функция get_good готова и ждёт текст на вход для генерации добра')
print (f'Функция get_bad готова и ждёт текст на вход для генерации зла')

Функция get_congr готова и ждёт текст на вход для генерации поздравления
Функция get_news готова и ждёт текст на вход для генерации упоротых новостей
